In [1]:
%pip install SentencePiece
%pip install sentencepiece
%pip install bert_score
%pip install evaluate
import nltk
nltk.download('punkt')
nltk.download('wordnet')
!pip install bert-score

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

english_text = "This is an example text."
input_text = f"Translate English to German: {english_text}"

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt")
# Generate the translation
output_ids = model.generate(input_ids)[0]

# Decode the generated ids to get the text
translation = tokenizer.decode(output_ids, skip_special_tokens=True)

print(f"English Text: {english_text}")
print(f"Translated Text: {translation}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

English Text: This is an example text.
Translated Text: Dies ist ein Beispieltext.


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [3]:
# read val_data
import pandas as pd
from tqdm import tqdm

val_data = pd.read_csv('val.csv')
test_data = pd.read_csv('test.csv')

# translate the english text to german using the model
translated_text_val = []
translated_text_test = []

for text in tqdm(val_data['en']):
    input_text = f"Translate English to German: {text}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output_ids = model.generate(input_ids)[0]
    translation = tokenizer.decode(output_ids, skip_special_tokens=True)
    translated_text_val.append(translation)

for text in tqdm(test_data['en']):
    input_text = f"Translate English to German: {text}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output_ids = model.generate(input_ids)[0]
    translation = tokenizer.decode(output_ids, skip_special_tokens=True)
    translated_text_test.append(translation)

# add the translated text to the dataframe
val_data_with_preds = val_data.copy()
# add new column to the dataframe
val_data_with_preds['pred_de'] = translated_text_val

test_data_with_preds = test_data.copy()
# add new column to the dataframe
test_data_with_preds['pred_de'] = translated_text_test

# save the translated data
val_data_with_preds.to_csv('val_translated_B.csv', index=False)
test_data_with_preds.to_csv('test_translated_B.csv', index=False)

  0%|          | 0/2169 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 2999/2999 [21:54<00:00,  2.28it/s]


In [4]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk import word_tokenize
import numpy as np

# Function to calculate BLEU scores
def calculate_bleu_scores(data):
    bleu_1_scores = []
    bleu_2_scores = []
    bleu_3_scores = []
    bleu_4_scores = []

    for index, row in data.iterrows():
        reference = [word_tokenize(row['de'].lower(), language='german')]
        candidate = word_tokenize(row['pred_de'].lower(), language='german')

        bleu_1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0), smoothing_function=SmoothingFunction().method1)
        bleu_2 = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0), smoothing_function=SmoothingFunction().method1)
        bleu_3 = sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0), smoothing_function=SmoothingFunction().method1)
        bleu_4 = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=SmoothingFunction().method1)

        bleu_1_scores.append(bleu_1)
        bleu_2_scores.append(bleu_2)
        bleu_3_scores.append(bleu_3)
        bleu_4_scores.append(bleu_4)

    return np.mean(bleu_1_scores), np.mean(bleu_2_scores), np.mean(bleu_3_scores), np.mean(bleu_4_scores)

# Calculate BLEU scores for the validation data
bleu_1, bleu_2, bleu_3, bleu_4 = calculate_bleu_scores(val_data_with_preds)
print("BLEU Scores for Validation Data")
print(f'BLEU-1 Score: {bleu_1}')
print(f'BLEU-2 Score: {bleu_2}')
print(f'BLEU-3 Score: {bleu_3}')
print(f'BLEU-4 Score: {bleu_4}')
print(f'Net BLEU Score: {(bleu_1 + bleu_2 + bleu_3 + bleu_4) / 4}')

print()

# Calculate BLEU scores for the test data
bleu_1, bleu_2, bleu_3, bleu_4 = calculate_bleu_scores(test_data_with_preds)
print("BLEU Scores for Test Data")
print(f'BLEU-1 Score: {bleu_1}')
print(f'BLEU-2 Score: {bleu_2}')
print(f'BLEU-3 Score: {bleu_3}')
print(f'BLEU-4 Score: {bleu_4}')
print(f'Net BLEU Score: {(bleu_1 + bleu_2 + bleu_3 + bleu_4) / 4}')

BLEU Scores for Validation Data
BLEU-1 Score: 0.34324355501112835
BLEU-2 Score: 0.24415199888229103
BLEU-3 Score: 0.18379732020241002
BLEU-4 Score: 0.1412590772111074
Net BLEU Score: 0.2281129878267342

BLEU Scores for Test Data
BLEU-1 Score: 0.35804893078722105
BLEU-2 Score: 0.26683363324252474
BLEU-3 Score: 0.20547564007268035
BLEU-4 Score: 0.15949951906993473
Net BLEU Score: 0.24746443079309022


In [5]:
import evaluate

# Assuming you have your 'de' and 'pred_de' lists from the DataFrame
references_val =  val_data_with_preds['de'].tolist()  # Your target translations
predictions_val = val_data_with_preds['pred_de'].tolist()  # Your predicted translations

meteor_score_val = evaluate.load("meteor")
results_val = meteor_score_val.compute(predictions=predictions_val, references=references_val)
print(f"METEOR Score on Validation Data: {results_val['meteor']}")

references_test =  test_data_with_preds['de'].tolist()  # Your target translations
predictions_test = test_data_with_preds['pred_de'].tolist()  # Your predicted translations

meteor_score_test = evaluate.load("meteor")
results_test = meteor_score_test.compute(predictions=predictions_test, references=references_test)
print(f"METEOR Score on Test Data: {results_test['meteor']}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


METEOR Score on Validation Data: 0.36057107894002044


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


METEOR Score on Test Data: 0.3836311927638204


In [6]:
predictions_val

['In Tokio treffen sich die Premierminister Indiens und Japans',
 'Der neue indische Premierminister Narendra Modi trifft sein japanisches Am',
 'Herr Modi ist auf einer fünftägigen Reise nach Japan, um die wirtschaftliche',
 'Auf der Tagesordnung stehen Pläne für eine verstärkte nukleare Zusammenarbeit.',
 'Indien hofft außerdem auf eine Vereinbarung über die Zusammenarbeit im Verteidigung',
 'Karratha Polizei verhaftet 20-jährige nach Hochgeschwindigkeits-Motor',
 'Ein Motorrad wurde beschlagnahmt, nachdem es 125 km/h',
 'Die Polizei der Verkehrsbehörden, die heute Morgen in Karratha auf der Patrouille',
 'Polizei: Der Fahrer hat dann nicht angehalten und fuhr weiter auf Burgess',
 'Das Motorrad und eine Person, die der Beschreibung des Fahrers entspricht, wurden dann in einem',
 'Karratha Police haben einen 20-jährigen Mann angeklagt, er sei',
 'Er wird am 23. September im Magistratgericht Karratha erscheinen.',
 'Das Motorrad wurde beschlagnahmt und drei Monate lang beschlagnahmt.'

In [7]:
references_val

['Die Premierminister Indiens und Japans trafen sich in Tokio.',
 'Indiens neuer Premierminister Narendra Modi trifft bei seinem ersten wichtigen Auslandsbesuch seit seinem Wahlsieg im Mai seinen japanischen Amtskollegen Shinzo Abe in Toko, um wirtschaftliche und sicherheitspolitische Beziehungen zu besprechen.',
 'Herr Modi befindet sich auf einer fünftägigen Reise nach Japan, um die wirtschaftlichen Beziehungen mit der drittgrößten Wirtschaftsnation der Welt zu festigen.',
 'Pläne für eine stärkere kerntechnische Zusammenarbeit stehen ganz oben auf der Tagesordnung.',
 'Berichten zufolge hofft Indien darüber hinaus auf einen Vertrag zur Verteidigungszusammenarbeit zwischen den beiden Nationen.',
 'Polizei von Karratha verhaftet 20-Jährigen nach schneller Motorradjagd',
 'Ein Motorrad wurde beschlagnahmt, nachdem der Fahrer es mit 125 km/h in einer 70 km/h-Zone und durch Buschland gefahren hatte, um der Polizei in Bilbara zu entkommen.',
 'Verkehrspolizisten in Karratha versuchten heu

In [8]:
bertscore = evaluate.load("bertscore")

results_val = bertscore.compute(predictions=predictions_val, references=references_val, lang="de")
print("BERTScore on Validation Data")
print(f"BERTScore Precision: {np.mean(results_val['precision'])}")
print(f"BERTScore Recall: {np.mean(results_val['recall'])}")
print(f"BERTScore F1: {np.mean(results_val['f1'])}")

print()

results_test = bertscore.compute(predictions=predictions_test, references=references_test, lang="de")
print("BERTScore on Test Data")
print(f"BERTScore Precision: {np.mean(results_test['precision'])}")
print(f"BERTScore Recall: {np.mean(results_test['recall'])}")
print(f"BERTScore F1: {np.mean(results_test['f1'])}")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BERTScore on Validation Data
BERTScore Precision: 0.8235960169671702
BERTScore Recall: 0.7633448714976928
BERTScore F1: 0.7911455879402688

BERTScore on Test Data
BERTScore Precision: 0.8340643794586675
BERTScore Recall: 0.7693550742121688
BERTScore F1: 0.7992029001649995
